<a href="https://colab.research.google.com/github/Bae-hong-seob/University-3-2/blob/main/Big_data_process_and_application/2_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 38.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=3f2407fa318fbd494e9d94eb5772c8a4cb9968e4346a210023cb59c8a769de2e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
!apt install openjdk-8-jdk-headless -qq

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155569 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u342-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u342-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
from operator import add

lines = spark.read.text("/content/drive/MyDrive/3-2 university/Big_data_process_and_application/word.txt").rdd.map(lambda r: r[0])
counts = lines.flatMap(lambda x: x.split(' ')).map(lambda x: (x,1)).reduceByKey(add)
output = counts.collect()

for (word, count) in output:
  print("%s: %i" % (word, count))

spark.stop()

i: 2
am: 1
happy: 1
keep: 1
going: 1
like: 1
cat: 1


# 9/19 수업내용

In [ ]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

SparkContext() 함수를 통해 spark와 연결.  
import pandas as pd 와 같은.

In [ ]:
data = sc.parallelize([('Amber',22), ('Alfred',23), ('Skye',4), ('Albert', 12), ('Amber', 20)])

pyspark.SparkContext 라이브러리 내의 parallelize 함수를 통해 data 저장.

In [ ]:
data.collect()

[('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 20)]

ㄴ collect()함수로 데이터 읽기.

## RDD는 schemaless data 구조이기 때문에 다양한 데이터 구조를 섞을 수 있다.

In [ ]:
data_heterogenous = sc.parallelize([('Ferrari', 'fast'), {'Porsche':100000}, ['Spain', 'visited', 4504]]).collect()
data_heterogenous

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]

ㄴ 보는 예시와 같이 list와 dictionary 데이터 구조를 섞을 수 있다.

In [ ]:
data_heterogenous[1]['Porsche']

100000

In [ ]:
data_heterogenous[2][0]

'Spain'

ㄴ 다음과 같이 출력 확인

## 기존 파일 데이터 읽어오기(read_csv) : textFile()함수 사용

In [ ]:
data_from_file = sc.textFile('/content/drive/MyDrive/3-2 university/Big_data_process_and_application/Iris.csv')

In [ ]:
data_from_file.take(5)

['SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species',
 '5.1,3.5,1.4,0.2,Iris-setosa',
 '4.9,3,1.4,0.2,Iris-setosa',
 '4.7,3.2,1.3,0.2,Iris-setosa',
 '4.6,3.1,1.5,0.2,Iris-setosa']

ㄴ data.head(5) 대신 spark에서는 take()함수 사용.

In [ ]:
# 헤더(첫줄) 추출
iris_header = data_from_file.first()

#spark RDD로 변환(header만 뗀 걸로 RDD 변환)
iris_header = sc.parallelize([iris_header])

#데이터 셋에서 헤더 제외
iris = data_from_file.subtract(iris_header)
iris.collect()[:10]

['4.6,3.4,1.4,0.3,Iris-setosa',
 '5,3.4,1.5,0.2,Iris-setosa',
 '4.9,3.1,1.5,0.1,Iris-setosa',
 '4.9,3.1,1.5,0.1,Iris-setosa',
 '4.9,3.1,1.5,0.1,Iris-setosa',
 '4.8,3.4,1.6,0.2,Iris-setosa',
 '5.7,3.8,1.7,0.3,Iris-setosa',
 '5.1,3.7,1.5,0.4,Iris-setosa',
 '5,3,1.6,0.2,Iris-setosa',
 '5.2,3.5,1.5,0.2,Iris-setosa']

1. data_from_file에서 첫줄(first)만 떼서 iris_header파일에 저장
2. iris_header파일을 parallelize함수를 사용하여 RDD로 변환
3. 기존 data_from_file (header 포함 RDD) 에서 헤더파일을 RDD로 변환시킨 iris_header를 빼기 연산 진행

## RDD(like DataFrame?) 제어 연산
Transformation 연산 : filter(), map()
- RDD에서 새로운 RDD를 생성하는 함수
- filter는 특정 데이터만 산출하여 새로운 RDD 타입 데이터 산출  
- map은 데이터를 분산 배치하여 새로운 RDD 타입 데이터 산출

Action 연산 : count(), collect()
- RDD에서 RDD가 아닌 다른 타입의 데이터로 변환하는 함수



### lambda 무명 함수(=익명 함수)
- lambda 매개변수 : 표현식
ex) lambda x : x+10
x에다 10을 더하는 익명함수 구현.  
이를 map() 함수와 연동한다면 데이터 구조에서 매 x(row) 값에 10을 더해줘라~

In [ ]:
iris_parsed_1 = iris.map(lambda row: row.split(","))
iris_parsed_1.take(10)

[['4.6', '3.4', '1.4', '0.3', 'Iris-setosa'],
 ['5', '3.4', '1.5', '0.2', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.8', '3.4', '1.6', '0.2', 'Iris-setosa'],
 ['5.7', '3.8', '1.7', '0.3', 'Iris-setosa'],
 ['5.1', '3.7', '1.5', '0.4', 'Iris-setosa'],
 ['5', '3', '1.6', '0.2', 'Iris-setosa'],
 ['5.2', '3.5', '1.5', '0.2', 'Iris-setosa']]

In [ ]:
iris_parsed_2 = iris_parsed_1.map(lambda x: ( str(x[-1]), float(x[0])))
iris_parsed_2.take(10)

[('Iris-setosa', 4.6),
 ('Iris-setosa', 5.0),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.8),
 ('Iris-setosa', 5.7),
 ('Iris-setosa', 5.1),
 ('Iris-setosa', 5.0),
 ('Iris-setosa', 5.2)]

In [ ]:
# filter() 함수를 사용하여 꽃받침 길이가 5.0 이상인 경우만 추출

#원본
iris_parsed_1.count()

150

In [ ]:
iris_filtered = iris_parsed_1.filter(lambda x: float(x[0]) >= 5.0)
# 필터링 완료.
iris_filtered.count()

128

flatMap()함수. map()함수 결과를 flat으로 표현. 즉 평평하게 1차원으로 output을 표현

In [ ]:
iris_flat = iris_parsed_1.flatMap(lambda x: ( str(x[-1]), float(x[0])))
iris_flat.take(6)

['Iris-setosa', 4.6, 'Iris-setosa', 5.0, 'Iris-setosa', 4.9]

In [ ]:
iris_flat = iris_parsed_1.flatMap(lambda x: (( str(x[-1]), float(x[0]))))
iris_flat.take(6)

['Iris-setosa', 4.6, 'Iris-setosa', 5.0, 'Iris-setosa', 4.9]

ㄴ 튜플로 구성해보려고 () 괄호로 한번 더 감싸주더라도 flatMap()함수는 결국 1차원으로 표현한다.

# 9/21 수업내용

distinct() 함수 사용하여 중복 제거.
- 새로운 RDD 생성, 즉 transformation 연산.

In [ ]:
distinct_iris = iris_parsed_1.map(lambda x: x[-1]).distinct().collect()
distinct_iris

['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']

sample() 함수
- sample(flase, 0.5, 123)
- 첫번째 인자 True:복원추출, False:비복원추출
- 두번째 인자 샘플의 비율
- 세번째 인자 seed값

In [ ]:
fraction = 0.5
iris_sample1 = iris_parsed_1.sample(False, fraction, 123)
iris_sample2 = iris_parsed_2.sample(False, fraction, 123)

#3번째 인자인 seed 값을 다르게 설정할 경우 다르게 sampling
iris_sample3 = iris_parsed_1.sample(False, fraction, 456)

iris_sample1.take(5)

[['5', '3.4', '1.5', '0.2', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.8', '3.4', '1.6', '0.2', 'Iris-setosa']]

In [ ]:
iris_sample2.take(5)

[('Iris-setosa', 5.0),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.8)]

In [ ]:
iris_sample3.take(5)

[['4.6', '3.4', '1.4', '0.3', 'Iris-setosa'],
 ['5', '3.4', '1.5', '0.2', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa']]

## join(transformation 연산) : 두 개의 RDD에서 같은 키를 가지는 요소끼리 조인

OutherJoin() : 외부조인. 즉 조인에 참여하지 않은 튜플까지 조인에 참여시킴

In [ ]:
rdd1 = sc.parallelize([('a',1),('b',4), ('c',10)])
rdd2 = sc.parallelize([('a',4),('a',1),('b',6),('d',15)])

rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, 6)), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

ㄴ 왼쪽 RDD에 오른쪽 RDD를 추가하는 방식으로 진행. (key값이 겹치는 것만 튜플 형식으로 추가)

In [ ]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('b', (4, 6)), ('a', (1, 4)), ('a', (1, 1))]

ㄴ 그냥 join()을 사용할 경우 두 RDD의 key값이 교집합만 출력(a,b)

In [ ]:
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

ㄴ intersection() 교집합을 반환하는 함수

## repartition() 함수

In [ ]:
len(rdd1.glom().collect())

2

In [ ]:
rdd1 = rdd1.repartition(4)
len(rdd1.glom().collect())

4

# Action 연산

-take(n) 함수는 RDD 전체가아니라 상위 n개만 출략(pd.battle pro shitttttttt


In [ ]:
iris_first = iris.take(1)
iris_first

['4.6,3.4,1.4,0.3,Iris-setosa']

reduce(리스트, 파티션 크기) 함수

리스트에서 몇개씩 진행될껀가..

In [ ]:
rdd1.collect()

[('a', 1), ('b', 4), ('c', 10)]

In [ ]:
rdd1.map(lambda row:row[1]).reduce(lambda x, y: x+y)

15

In [ ]:
data_reduce = sc.parallelize([1,2,.5,.1,5,.2],1)
data_reduce.reduce(lambda x,y : x/y)

10.0

In [ ]:
data_reduce = sc.parallelize([1,2,.5,.1,5,.2],3)
data_reduce.reduce(lambda x,y : x/y)

0.004

reduceByKey() 함수
- 자주사용함. 추후에 알아보기

saveAsTextFile() 함수
- RDD의 내용을 파일로 출력
- 같은 디렉토리에 파티션 단위로 각 파일을 따로 출력

# 추가 학습 진행. 중요함

Action연산
- mapValues() 함수는 key의 변경 없이 각 value에 함수 적용
- 이전 실습 때 종과 꽃받침 길이만 추출했던 rdd 사용(=iris_parsed_2)

In [ ]:
iris_parsed_2 = iris_parsed_1.map(lambda x:( str(x[-1]), float(x[0])))
iris_parsed_2.take(10)

[('Iris-setosa', 4.6),
 ('Iris-setosa', 5.0),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.9),
 ('Iris-setosa', 4.8),
 ('Iris-setosa', 5.7),
 ('Iris-setosa', 5.1),
 ('Iris-setosa', 5.0),
 ('Iris-setosa', 5.2)]

- 종 별 개수와 꽃받침 길이 값을 합산하여 나타내기 위해 mapValues()를 사용하여 각 꽃받침 길이와 숫자1을 저장하는 rdd 생성

In [ ]:
# 종 별 갯수와 꽃받침 길이 값을 합산하여 나타내기
iris_totals = iris_parsed_2.mapValues(lambda x: (x,1))
iris_totals.take(10)

[('Iris-setosa', (4.6, 1)),
 ('Iris-setosa', (5.0, 1)),
 ('Iris-setosa', (4.9, 1)),
 ('Iris-setosa', (4.9, 1)),
 ('Iris-setosa', (4.9, 1)),
 ('Iris-setosa', (4.8, 1)),
 ('Iris-setosa', (5.7, 1)),
 ('Iris-setosa', (5.1, 1)),
 ('Iris-setosa', (5.0, 1)),
 ('Iris-setosa', (5.2, 1))]

In [ ]:
# 각 종 별 평균 계산을 위하여 reduceByKey()함수 사용하여 합산처리 후 평균 계산
iris_totals = iris_totals.reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1]))
iris_totals.collect()

[('Iris-setosa', (250.29999999999998, 50)),
 ('Iris-virginica', (329.40000000000003, 50)),
 ('Iris-versicolor', (296.79999999999995, 50))]

In [ ]:
# 종 별 꽃받침 길이의 평균
iris_means = iris_totals.mapValues(lambda x: (x[0] / x[1]))
result = iris_means.collect()
result

[('Iris-setosa', 5.005999999999999),
 ('Iris-virginica', 6.588000000000001),
 ('Iris-versicolor', 5.935999999999999)]

In [ ]:
#print
for val in result:
  print(f"iris species {val[0]} has an average sepal length of {val[1]}")

iris species Iris-setosa has an average sepal length of 5.005999999999999
iris species Iris-virginica has an average sepal length of 6.588000000000001
iris species Iris-versicolor has an average sepal length of 5.935999999999999


sortBy() 함수
- 두번째 인자 False:내림차순, True(defalut):오름차순

In [ ]:
#정렬
iris_sort = iris_means.sortBy(lambda x: x[1], False)
iris_sort.collect()

[('Iris-virginica', 6.588000000000001),
 ('Iris-versicolor', 5.935999999999999),
 ('Iris-setosa', 5.005999999999999)]